In [1]:
import torch
import pytorch_ssim

In [2]:
from swin_wnet_model import SwinWnet
from utilities import MAE_BCE_Loss, visualize_density_maps, calculate_count_metrics

In [3]:
torch.cuda.device_count()

1

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
model = SwinWnet()
model.load_state_dict(torch.load('model.pth'))
if torch.cuda.is_available() and torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
model = model.cuda()

criterion = MAE_BCE_Loss()

In [6]:
def evaluate(model, test_loader, criterion, sigma=0):
    model.eval()
    test_running_loss = 0.0
    test_running_mae = 0.0
    test_running_mse = 0.0
    test_running_rmse = 0.0

    with torch.no_grad():
        for images, target_density_maps, target_reinforcement_maps in test_loader:
            images, target_density_maps, target_reinforcement_maps = images.cuda(), target_density_maps.cuda(), target_reinforcement_maps.cuda()

            generated_density_maps, generated_reinforcement_maps = model(images)

            loss = criterion(generated_density_maps, generated_reinforcement_maps, target_density_maps, target_reinforcement_maps)
            test_running_loss += loss.item()

            generated_density_maps = torch.relu(generated_density_maps)
#             visualize_density_maps(images, target_density_maps, generated_density_maps, sigma=sigma)
            

            mae, mse, rmse = calculate_count_metrics(generated_density_maps, target_density_maps)
            test_running_mae += mae
            test_running_mse += mse
            test_running_rmse += rmse

    avg_test_loss = test_running_loss / len(test_loader)
    avg_test_mae = test_running_mae / len(test_loader)
    avg_test_mse = test_running_mse / len(test_loader)
    avg_test_rmse = test_running_rmse / len(test_loader)

    print(f"Test - Loss: {avg_test_loss:.5f}, MAE: {avg_test_mae:.5f}, MSE: {avg_test_mse:.5f}, RMSE: {avg_test_rmse:.5f}")

### QNRF Dataset

In [7]:
from QNRF_Dataloader import prepare_dataloaders
train_dir = "../../UCF-QNRF_ECCV18/Train"
test_dir = "../../UCF-QNRF_ECCV18/Test"
qnrf_train_loader, qnrf_val_loader, qnrf_test_loader = prepare_dataloaders(train_dir=train_dir, test_dir=test_dir, val_split=0.2)

In [8]:
evaluate(model, qnrf_test_loader, criterion,sigma=0)

Test - Loss: 11.46678, MAE: 260.48846, MSE: 197807.89041, RMSE: 260.48846


### ShanghaiTech A Dataset

In [9]:
from ShanghaiTechA_Dataloader import prepare_dataloaders
train_image_path = '../../ShanghaiTech_Crowd_Counting_Dataset/part_A_final/train_data/images'
train_annotation_path = '../../ShanghaiTech_Crowd_Counting_Dataset/part_A_final/train_data/ground_truth'
test_image_path = '../../ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data/images'
test_annotation_path = '../../ShanghaiTech_Crowd_Counting_Dataset/part_A_final/test_data/ground_truth'
STA_train_loader, STA_val_loader, STA_test_loader = prepare_dataloaders(train_image_path, train_annotation_path, test_image_path, test_annotation_path)

In [10]:
evaluate(model, STA_test_loader, criterion, sigma=0)

Test - Loss: 8.86193, MAE: 140.26835, MSE: 53771.30998, RMSE: 140.26835


### UCF-CC-50 Dataset

In [11]:
from CC_50_Dataloader import prepare_dataloaders
test_dir = "../../UCF_CC_50/"
cc_50_test_loader = prepare_dataloaders(test_dir=test_dir)

In [12]:
evaluate(model, cc_50_test_loader, criterion, sigma=0)

Test - Loss: 21.23085, MAE: 647.64769, MSE: 1214567.65187, RMSE: 647.64769
